In [21]:
import os
import re
import logging
from collections import defaultdict

import numpy as np
# numpy bool alias compatibility for older code that expects np.bool
if not hasattr(np, "bool"):
    np.bool = np.bool_

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

from backbones.iresnet import iresnet50
from datasets import CasiaWebFace, BinDataset

logging.basicConfig(level=logging.DEBUG, filename="influence_verification.log", filemode="w")
logger = logging.getLogger(__name__)

In [22]:
WEIGHTS_DIR = "/home/seidel/r50_webface_COSFace035"
CASIA_ROOT = "/data/ozgur/casia_training"
CASIA_NUM_CLASSES = 10572
VAL_DIR = "/home/seidel/validation"
VAL_TARGETS = [
    "lfw",
    "cfp_fp",
    "cfp_ff",
    "agedb_30",
    "cplfw",
    "calfw",
]

OUTPUT_GRADS_DIR = "output/gradients"
OUTPUT_INFL_DIR = "output/tracin"

In [23]:
# ----------------------
# Subset Configuration
# ----------------------
# When set to an integer, only the first NUM_TRAIN_CLASSES identities will be used from CASIA.
# Set to None to use the full training set (slower).
NUM_TRAIN_CLASSES = 100  # change this number for larger/smaller subsets
print(f"Using NUM_TRAIN_CLASSES = {NUM_TRAIN_CLASSES}")

Using NUM_TRAIN_CLASSES = 100


In [ ]:
def contrastive_loss(features1, features2, labels, margin=0.5):
    """Scalar contrastive loss for a batch or single pair.

    Args:
        features1, features2: [B, D] embeddings
        labels: [B] with values 1 (same) or 0 (different)
        margin: margin for negative pairs
    Returns: scalar loss (mean over batch)
    """
    euclidean_distance = F.pairwise_distance(features1, features2)
    loss = labels * euclidean_distance.pow(2) + (1 - labels) * (
        torch.clamp(margin - euclidean_distance, min=0.0).pow(2)
    )
    return loss.mean()


def flatten_grads(grad_list):
    return torch.cat([g.reshape(-1) for g in grad_list if g is not None])


def _pair_grad(backbone, img1, img2, label01, device):
    """Compute flattened gradient vector for a single pair loss w.r.t. backbone params."""
    backbone.zero_grad(set_to_none=True)
    img1 = img1.unsqueeze(0).to(device, non_blocking=True)
    img2 = img2.unsqueeze(0).to(device, non_blocking=True)
    # ensure label tensor type matches computation dtype
    lbl = torch.as_tensor(label01, device=device, dtype=img1.dtype).view(1)
    f1 = backbone(img1)
    f2 = backbone(img2)
    loss = contrastive_loss(f1, f2, lbl)
    grads = torch.autograd.grad(
        loss,
        [p for p in backbone.parameters() if p.requires_grad],
        retain_graph=False,
        create_graph=False,
        allow_unused=True,
    )
    # log shapes for debugging
    flat = flatten_grads(grads).detach().cpu()
    return flat


# ----------------------
# Checkpoint utilities
# ----------------------


def select_evenly_spaced_checkpoints(filenames, num_select):
    """Support filenames like 'backbone_31614.pth' or '31614backbone.pth'."""
    def extract_iter(f):
        m = re.search(r"(\d+)", f)
        return int(m.group(1)) if m else -1

    files_sorted = sorted(filenames, key=extract_iter)
    if num_select >= len(files_sorted):
        return files_sorted
    idxs = np.linspace(0, len(files_sorted) - 1, num=num_select, dtype=int)
    idxs = sorted(set(map(int, idxs.tolist())))
    while len(idxs) < num_select:
        for i in range(len(files_sorted)):
            if i not in idxs:
                idxs.append(i)
                if len(idxs) == num_select:
                    break
    idxs.sort()
    return [files_sorted[i] for i in idxs]


def load_backbone_weights(backbone, checkpoint_path, device):
    """Load backbone weights from a checkpoint that may store raw state_dict or a dict."""
    state = torch.load(checkpoint_path, map_location=device)
    backbone.load_state_dict(state)
    backbone.eval()

    # only compute gradients for weights of fully connected layer (TracIn approximation)
    for p in backbone.parameters():
        p.requires_grad_(False)
        
    try:
        backbone.fc.requires_grad_(True)
    except Exception:
        # If architecture names differ, enable grads for last parameter group
        for p in list(backbone.parameters())[-2:]:
            p.requires_grad_(True)

    learning_rate = state.get('learning_rate', 1.0)
    return learning_rate

# ----------------------
# Train pairs and gradient caching
# ----------------------

def _pairs_from_batch(images, labels, imageindices, max_pairs_per_batch=32):
    """Return a small, balanced list of (i, j, y01, img_idx_i, img_idx_j)."""
    by_label = defaultdict(list)
    for i, y in enumerate(labels.tolist()):
        by_label[y].append(i)

    pos, neg = [], []
    # positive pairs (consecutive within each label)
    for idxs in by_label.values():
        if len(idxs) > 1:
            for k in range(len(idxs) - 1):
                i, j = idxs[k], idxs[k + 1]
                pos.append((i, j, 1, int(imageindices[i]), int(imageindices[j])))

    # negative pairs (simple cross-label sampling)
    labs = list(by_label.keys())
    for a in range(len(labs)):
        if not by_label[labs[a]]:
            continue
        i = by_label[labs[a]][0]
        b = len(labs) - 1 - a
        if b < 0:
            continue
        if by_label[labs[b]]:
            j = by_label[labs[b]][0]
            if labels[i] != labels[j]:
                neg.append((i, j, 0, int(imageindices[i]), int(imageindices[j])))

    # balance and cap
    pairs = []
    p_needed = max_pairs_per_batch // 2
    n_needed = max_pairs_per_batch - p_needed
    pairs.extend(pos[:p_needed])
    pairs.extend(neg[:n_needed])
    return pairs


def calc_grads_train(
    checkpoints,
    backbone,
    device,
    max_train_pairs_per_ckpt=200,
    batch_size=32,
    num_workers=4,
    num_classes=None,
):
    """Compute and cache gradient vectors for a subset of train pairs per checkpoint."""
    if num_classes is None:
        use_num_classes = CASIA_NUM_CLASSES
    else:
        use_num_classes = num_classes
    train_dataset = CasiaWebFace(CASIA_ROOT, local_rank=0, num_classes=use_num_classes)
    train_dl = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    os.makedirs(OUTPUT_GRADS_DIR, exist_ok=True)

    with torch.autograd.set_grad_enabled(True):
        for ci, cp in enumerate(checkpoints):
            ckpt_path = os.path.join(WEIGHTS_DIR, cp) if not os.path.isabs(cp) else cp
            print(f"[{ci+1}/{len(checkpoints)}] Loading checkpoint: {ckpt_path}")
            load_backbone_weights(backbone, ckpt_path, device)

            saved = 0
            for images, labels, imageindices in tqdm(train_dl, desc=f"train-pairs@{cp}", total=max_train_pairs_per_ckpt//batch_size*2):
                images = images.to(device)
                labels = labels.to(device)

                pairs = _pairs_from_batch(images, labels, imageindices, max_pairs_per_batch=32)
                for i, j, y01, gi, gj in pairs:
                    grad_vec = _pair_grad(
                        backbone,
                        images[i].detach().cpu(),
                        images[j].detach().cpu(),
                        y01,
                        device,
                    )
                    save_path = os.path.join(
                        OUTPUT_GRADS_DIR, f"{os.path.basename(cp)}__pair_{saved:07d}.pt"
                    )
                    torch.save(
                        {
                            "ckpt": os.path.basename(cp),
                            "train_img_i": gi,
                            "train_img_j": gj,
                            "label01": int(y01),
                            "grad": grad_vec,
                        },
                        save_path,
                    )
                    saved += 1
                    if saved >= max_train_pairs_per_ckpt:
                        break
                if saved >= max_train_pairs_per_ckpt:
                    break

def lr_step_func(epoch):
        return ((epoch + 1) / (4 + 1)) ** 2 if epoch < -1 else 0.1 ** len(
            [m for m in [20, 28, 32] if m - 1 <= epoch])

# ----------------------
# TracIn influence computation
# ----------------------

def tracin_influence_for_test_pair(backbone, cp, test_pair, label_bool, learning_rate, device):
    """Compute per-training-image influence for one test pair at one checkpoint.

    Returns: dict {train_image_index: influence_value}
    """
    # Compute test gradient once
    img1, img2 = test_pair
    lbl01 = 1 if bool(label_bool) else 0
    test_grad = _pair_grad(backbone, img1, img2, lbl01, device)

    # Determine checkpoint learning rate
    checkpoint_path = os.path.join(WEIGHTS_DIR, cp) if not os.path.isabs(cp) else cp

    # Accumulate dot products per training image by streaming over saved gradients
    infl = defaultdict(float)
    grad_files = [
        os.path.join(OUTPUT_GRADS_DIR, f)
        for f in os.listdir(OUTPUT_GRADS_DIR)
        if f.startswith(f"{os.path.basename(cp)}__pair_")
    ]

    for gf in grad_files:
        rec = torch.load(gf)
        g = rec["grad"]
        dot = float(torch.dot(test_grad, g)) * float(learning_rate)
        # assign to both images in the pair
        infl[rec["train_img_i"]] += dot * 0.5
        infl[rec["train_img_j"]] += dot * 0.5
    return infl


def save_topk_influence(infl_map, out_csv_path, topk=100):
    os.makedirs(os.path.dirname(out_csv_path), exist_ok=True)
    # Sort by absolute influence desc
    items = sorted(infl_map.items(), key=lambda kv: abs(kv[1]), reverse=True)[:topk]
    with open(out_csv_path, "w") as f:
        f.write("train_image_index,influence\n")
        for idx, val in items:
            f.write(f"{idx},{val}\n")


In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Resolve checkpoints
all_weights = [
    f
    for f in os.listdir(WEIGHTS_DIR)
    if os.path.isfile(os.path.join(WEIGHTS_DIR, f)) and "backbone" in f and f.endswith(".pth")
]
backbone_weights = select_evenly_spaced_checkpoints(all_weights, 8)

backbone = iresnet50().to(device)

# Precompute and cache training pair gradients per checkpoint (pairwise loss, no header)
need_train_grads = not os.path.exists(OUTPUT_GRADS_DIR) or not os.listdir(OUTPUT_GRADS_DIR)
if need_train_grads:
    print("No existing gradients found. Calculating training pair gradients...")
    calc_grads_train(backbone_weights, backbone, device)
else:
    print("Using existing training pair gradients.")

# For each validation set and checkpoint, compute per-image TracIn
for d in VAL_TARGETS:
    test_dataset = BinDataset(os.path.join(VAL_DIR, d + ".bin"))
    indices = torch.arange(100)  # Use only first 100 pairs for testing
    test_dataset = Subset(test_dataset, indices)
    test_dl = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

    for cp in backbone_weights:
        ckpt_path = os.path.join(WEIGHTS_DIR, cp) if not os.path.isabs(cp) else cp
        print(f"Eval TracIn on {d} @ {ckpt_path}")
        load_backbone_weights(backbone, ckpt_path, device)

        out_dir = os.path.join(OUTPUT_INFL_DIR, d, os.path.basename(cp))
        os.makedirs(out_dir, exist_ok=True)

        optimizer = torch.optim.SGD(params=[{'params': backbone.parameters()}], lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.LambdaLR(
            optimizer=optimizer, lr_lambda=lr_step_func)
        learning_rate = scheduler.get_last_lr()[0]
        print(f"Using learning rate: {learning_rate}")

        for pair_idx, (orig_pair, flipped_pair, label_bool) in enumerate(
            tqdm(test_dl, desc=f"{d}@{cp}")
        ):
            # Use original pair only (optionally average with flipped)
            img1, img2 = orig_pair
            infl_map = tracin_influence_for_test_pair(
                backbone, cp, (img1[0], img2[0]), label_bool[0].item(), learning_rate, device
            )
            save_topk_influence(
                infl_map,
                os.path.join(out_dir, f"pair_{pair_idx:06d}.csv"),
                topk=100,
            )


Using existing training pair gradients.
loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
torch.Size([12000, 3, 112, 112])
num of pairs:  6000
Eval TracIn on lfw @ /home/seidel/r50_webface_COSFace035/958backbone.pth


lfw@958backbone.pth:   0%|          | 0/100 [00:00<?, ?it/s]

Using learning rate: 0.1


lfw@958backbone.pth:   1%|          | 1/100 [00:04<07:48,  4.74s/it]

Using learning rate: 0.1


...

calfw@31614backbone.pth:  98%|█████████▊| 98/100 [08:48<00:09,  4.51s/it]

Using learning rate: 0.1


calfw@31614backbone.pth:  99%|█████████▉| 99/100 [08:52<00:04,  4.51s/it]

Using learning rate: 0.1


calfw@31614backbone.pth: 100%|██████████| 100/100 [08:58<00:00,  5.38s/it]


In [ ]:
# Parameters
d = VAL_TARGETS[0]
cp = backbone_weights[0]
topk = 20

# Prepare dataset and loader (batch_size=1 as elsewhere)
test_dataset = BinDataset(os.path.join(VAL_DIR, d + ".bin"))
test_dl = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

# Load checkpoint into backbone
ckpt_path = os.path.join(WEIGHTS_DIR, cp) if not os.path.isabs(cp) else cp
print(f"Using val set '{d}' and checkpoint '{cp}'")
learning_rate = load_backbone_weights(backbone, ckpt_path, device)

# Grab the first pair
try:
    pair_iter = iter(test_dl)
    orig_pair, flipped_pair, label_bool = next(pair_iter)
except StopIteration:
    raise RuntimeError(f"Validation dataset {d} is empty")

img1, img2 = orig_pair

infl_map = tracin_influence_for_test_pair(backbone, cp, (img1[0], img2[0]), label_bool[0].item(), learning_rate, device)

if not infl_map:
    print("No precomputed training gradients found for this checkpoint (OUTPUT_GRADS_DIR may be empty).")
else:
    # Sort by influence value (ascending = most negative first)
    items_sorted_asc = sorted(infl_map.items(), key=lambda kv: kv[1])
    items_sorted_desc = list(reversed(items_sorted_asc))

    proponents = items_sorted_asc[:topk]  # most negative influence (lower test loss)
    opponents = items_sorted_desc[:topk]  # most positive influence (raise test loss)

    print(f"Top {topk} proponents (train_index, influence) - most lowering first:")
    for idx, val in proponents:
        print(f"  {idx}: {val}")

    print(f"\nTop {topk} opponents (train_index, influence) - most raising first:")
    for idx, val in opponents:
        print(f"  {idx}: {val}")

    # Save CSV with both lists to OUTPUT_INFL_DIR/<d>/<cp>/pair_000000_proponents_opponents.csv
    out_dir = os.path.join(OUTPUT_INFL_DIR, d, os.path.basename(cp))
    os.makedirs(out_dir, exist_ok=True)
    out_csv = os.path.join(out_dir, "pair_000000_proponents_opponents.csv")
    with open(out_csv, "w") as f:
        f.write("type,train_image_index,influence\n")
        for idx, val in proponents:
            f.write(f"proponent,{idx},{val}\n")
        for idx, val in opponents:
            f.write(f"opponent,{idx},{val}\n")

    print(f"Saved proponents/opponents CSV to: {out_csv}")


loading bin 0
loading bin 1000
loading bin 1000
loading bin 2000
loading bin 2000
loading bin 3000
loading bin 3000
loading bin 4000
loading bin 4000
loading bin 5000
loading bin 5000
loading bin 6000
loading bin 6000
loading bin 7000
loading bin 7000
loading bin 8000
loading bin 8000
loading bin 9000
loading bin 9000
loading bin 10000
loading bin 10000
loading bin 11000
loading bin 11000
torch.Size([12000, 3, 112, 112])
num of pairs:  6000
Using val set 'lfw' and checkpoint '958backbone.pth'
torch.Size([12000, 3, 112, 112])
num of pairs:  6000
Using val set 'lfw' and checkpoint '958backbone.pth'
Top 20 proponents (train_index, influence) - most lowering first:
  428322: -13050.189453125
  428380: -13050.189453125
  233713: -12003.1513671875
  233886: -12003.1513671875
  44904: -6841.263671875
  44853: -6841.263671875
  102508: -584.2177124023438
  102514: -584.2177124023438
  217914: 0.0
  308679: 0.0
  133401: 0.0
  135600: 0.0
  33919: 0.0
  404400: 0.0
  74614: 0.0
  107690: 0.0
  